In [2]:
#Port from Adventures in OpenCL Part1 to PyOpenCL
# http://enja.org/2010/07/13/adventures-in-opencl-part-1-getting-started/
# http://documen.tician.de/pyopencl/

# >> N = 1e8; a=0:N; b=0:N; tic; sin(a.*b); toc;
# Elapsed time is 10.165289 seconds.

import pyopencl as cl
import numpy
import time 
import numpy as np

N = int(1e8)

class CL:
    def __init__(self):
        self.ctx = cl.create_some_context()
        self.queue = cl.CommandQueue(self.ctx)

    def loadProgram(self, filename):
        #read in the OpenCL source file as a string
        fstr = """__kernel void part1(__global float* a, __global float* b, __global float* c)
        {
            unsigned int i = get_global_id(0);

            c[i] = sin(a[i] * b[i]);
        }"""
        #create the program
        self.program = cl.Program(self.ctx, fstr).build()

    def popCorn(self):
        mf = cl.mem_flags

        #initialize client side (CPU) arrays
        self.a = numpy.array(range(N), dtype=numpy.float32)
        self.b = numpy.array(range(N), dtype=numpy.float32)

        #create OpenCL buffers
        self.a_buf = cl.Buffer(self.ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=self.a)
        self.b_buf = cl.Buffer(self.ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=self.b)
        self.dest_buf = cl.Buffer(self.ctx, mf.WRITE_ONLY, self.b.nbytes)

    def execute(self):
        self.program.part1(self.queue, self.a.shape, None, self.a_buf, self.b_buf, self.dest_buf)
        c = numpy.empty_like(self.a)
        cl.enqueue_copy(self.queue, self.dest_buf, c).wait()
        #print("a", self.a)
        #print("b", self.b)
        #print( "c", c)



if __name__ == "__main__":
    example = CL()
    example.loadProgram("part1.cl")
    t_0 = time.time()
    example.popCorn()
    t_pop = time.time()-t_0
    print('t_pop', t_pop)
    
    t_0 = time.time()
    example.execute()
    t_gpu = time.time()-t_0
    
    print(t_gpu)
    
    
    a = numpy.array(range(N), dtype=numpy.float32)
    b = numpy.array(range(N), dtype=numpy.float32)
    t_0 = time.time()
    np.sin(a*b)
    print(time.time()-t_0)
    

ImportError: No module named 'pyopencl'